In [ ]:
import pandas as pa
import spacy
from fuzzywuzzy import process

/usr/local/python/3.12.1/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [8]:
from mappings import *

In [9]:
open_f = "datasets"
save_f = "transformed_data"

In [ ]:
init_soil_data = pa.read_csv(f"{open_f}/soil.csv")
init_irrigated_area_data = pa.read_csv(f"{open_f}/irrigated_area.csv")
init_price_data = pa.read_csv(f"{open_f}/crop_prices.csv")
init_weather_data = pa.read_csv(f"{open_f}/weather.csv")

# Irrigated Area

In [4]:
def transform_irrigated_area_data(df):
    df_2020 = df[df['Year'] == 2020]
    df_2020['Address'] = df_2020['State Code'].astype(str) + ', ' + df_2020['State Name'] + ', ' + df_2020['Dist Name']

    # df_2020 = df_2020[['Dddist Code', 'Address'] + [col for col in df.columns if 'IRRIGATED AREA' in col]]
    df_2020['Total Area'] = df_2020[[col for col in df_2020.columns if 'IRRIGATED AREA' in col]].sum(axis=1)

    df_final = df_2020[['Dist Code', 'Address', 'Total Area']]
    df_final.to_csv(f'{save_f}/irrigated_areacsv', index=False)
    print("Transformation for irrigated area complete.")

# Soil nutrients

In [5]:
def transform_soil_nutrient_levels(df):
    macronutrient_categories = ["Nitrogen", "Phosphorous", "Potassium", "OC", "EC", "pH"]
    micronutrient_categories = ["Copper", "Boron", "S", "Fe", "Zn", "Mn"]
    
    combined_results = {}

    for nutrient in macronutrient_categories:
        columns = [col for col in df.columns if col.startswith(nutrient)]
        def determine_macronutrient_level(row):
            values = row[columns]
            max_index = values.idxmax()
            return max_index.split(" - ")[1]
        combined_results[nutrient] = df.apply(determine_macronutrient_level, axis=1)

    for nutrient in micronutrient_categories:
        columns = [col for col in df.columns if col.startswith(nutrient)]
        def determine_micronutrient_level(row):
            values = row[columns]
            max_index = values.idxmax()
            return max_index.split(" - ")[1]
        combined_results[nutrient] = df.apply(determine_micronutrient_level, axis=1)

    combined_df = pa.DataFrame(combined_results)

    other_columns = ['District'] 
    df_final = pa.concat([df[other_columns], combined_df], axis=1)

    df_final.to_csv(f'{save_f}/soil.csv', index=False)
    print("Transformation for Soil Data complete.")

# Prices

In [13]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 24.7 MB/s eta 0:00:0000:010:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [14]:
nlp = spacy.load("en_core_web_sm")

standard_commodities = [
    'RICE', 'WHEAT', 'KHARIF SORGHUM', 'RABI SORGHUM', 'SORGHUM', 
    'PEARL MILLET', 'MAIZE', 'FINGER MILLET', 'BARLEY', 'CHICKPEA', 
    'PIGEONPEA', 'MINOR PULSES', 'PULSES', 'GROUNDNUT', 'SESAMUM', 
    'LINSEED', 'SUGARCANE', 'COTTON', 'FRUITS AND VEGETABLES', 'FODDER'
]

def match_commodity_entity(commodity, variety):
    combined_text = f"{commodity} {variety}"
    doc = nlp(combined_text)
    entities = [ent.text.upper() for ent in doc.ents]
    if entities:
        best_match = process.extractOne(entities[0], standard_commodities)
        return best_match[0]
    
    return process.extractOne(combined_text.upper(), standard_commodities)[0]

def transform_crop_price(df):
    df = df.drop(columns=['Market', 'Grade', 'Arrival_Date', 'Modal_x0020_Price'])
    df['Commodity'] = df.apply(lambda row: match_commodity_entity(row['Commodity'], row['Variety']), axis=1)
    return df

# TRANFORMM!!!!